In [ ]:
#MASK


# measure the points along the axis since I thought that it was in galactic coordinates so I missnamed them with lon and lat
lon=np.repeat(np.linspace(115,250,135*10),6*10)
lat=np.tile(np.linspace(24,30,6*10),135*10)
# transform the radec coordinates
eq = SkyCoord(lon, lat, unit='deg')
l, b = eq.galactic.l.value, eq.galactic.b.value
#transform into the pixel indices
notmask_pixel_1=hp.ang2pix(512,l,b,lonlat=True)

#repeat for other areas
lon=np.repeat(np.linspace(140,230,90*10),2*10)
lat=np.tile(np.linspace(20,22,2*10),90*10)
eq = SkyCoord(lon, lat, unit='deg')
l, b = eq.galactic.l.value, eq.galactic.b.value
notmask_pixel_2_1=hp.ang2pix(512,l,b,lonlat=True)

lon=np.repeat(np.linspace(130,230,100*10),2*10)
lat=np.tile(np.linspace(18,22,2*10),100*10)
eq = SkyCoord(lon, lat, unit='deg')
l, b = eq.galactic.l.value, eq.galactic.b.value
notmask_pixel_2_2=hp.ang2pix(512,l,b,lonlat=True)

lon=np.repeat(np.linspace(115,250,135*10),18*10)
lat=np.tile(np.linspace(0,18,18*10),135*10)
eq = SkyCoord(lon, lat, unit='deg')
l, b = eq.galactic.l.value, eq.galactic.b.value
notmask_pixel_2_3=hp.ang2pix(512,l,b,lonlat=True)

lon=np.repeat(np.linspace(-30,45,75*10),10*10)
lat=np.tile(np.linspace(22,32,10*10),75*10)
eq = SkyCoord(lon, lat, unit='deg')
l, b = eq.galactic.l.value, eq.galactic.b.value
notmask_pixel_3=hp.ang2pix(512,l,b,lonlat=True)

lon=np.repeat(np.linspace(-30,45,75*10),2*10)
lat=np.tile(np.linspace(14,16,2*10),75*10)
eq = SkyCoord(lon, lat, unit='deg')
l, b = eq.galactic.l.value, eq.galactic.b.value
notmask_pixel_4=hp.ang2pix(512,l,b,lonlat=True)

lon=np.repeat(np.linspace(-30,45,75*10),4*10)
lat=np.tile(np.linspace(6,10,4*10),75*10)
eq = SkyCoord(lon, lat, unit='deg')
l, b = eq.galactic.l.value, eq.galactic.b.value
notmask_pixel_5=hp.ang2pix(512,l,b,lonlat=True)

lon=np.repeat(np.linspace(-30,45,75*10),2*10)
lat=np.tile(np.linspace(0,2,2*10),75*10)
eq = SkyCoord(lon, lat, unit='deg')
l, b = eq.galactic.l.value, eq.galactic.b.value
notmask_pixel_6=hp.ang2pix(512,l,b,lonlat=True)

#make the mask
m[notmask_pixel_1]=hp.UNSEEN
m[notmask_pixel_2_1]=hp.UNSEEN
m[notmask_pixel_2_2]=hp.UNSEEN
m[notmask_pixel_2_3]=hp.UNSEEN
m[notmask_pixel_3]=hp.UNSEEN
m[notmask_pixel_4]=hp.UNSEEN
m[notmask_pixel_5]=hp.UNSEEN
m[notmask_pixel_6]=hp.UNSEEN

#because we want the pixels to be TRUE so i used mask_bad function
hi_mask=hp.mask_bad(m)

density_masked=hp.ma(m)
density_masked.mask = np.logical_not(hi_mask)

hp.mollview(density)
hp.mollview(density_masked.filled())

In [ ]:
#jackknife

from astropy.stats import jackknife_stats
import random
#example: divide up the space in map 13 (the map with HI mass>10^10.4) for 20 bins
n_bins=20

#all the pixels of masked area
mask=np.array([])
mask=np.append(mask,notmask_pixel_1)
mask=np.append(mask,notmask_pixel_2_1)
mask=np.append(mask,notmask_pixel_2_2)
mask=np.append(mask,notmask_pixel_2_3)
mask=np.append(mask,notmask_pixel_3)
mask=np.append(mask,notmask_pixel_4)
mask=np.append(mask,notmask_pixel_5)
mask=np.append(mask,notmask_pixel_6)

#randomly choose the pixels to construct the bins
r_mask=np.arange(mask.size)#get the indices of array "mask"
n_per_bin=int(mask.size/n_bins)#number of pixels per bin
for i in range(n_bins):
    indices=np.random.choice(r_mask,n_per_bin)
    globals()['map13_masks_%s' % (i+1)] = np.array(mask[indices])
    mask=np.delete(mask,indices)
    r_mask=np.arange(mask.size)
                                                    
#put on the mask of different bins
for i in range(n_bins):
    for j in range(globals()['map13_masks_%s' % (i+1)].size):
        k=int(globals()['map13_masks_%s' % (i+1)][j])
        mass_map_mask13[k]=hp.UNSEEN
    region_mask=hp.mask_bad(mass_map_mask13)
    globals()['map13_region_%s' % (i+1)]=hp.ma(mass_map_mask13)
    globals()['map13_region_%s' % (i+1)].mask = np.logical_not(region_mask)

#calculate the power spectrum of each region and its jackknife bias
jack_knife=np.array([])
test_statistic=np.var
for i in range(n_bins):
    globals()['cl_13_region_%s' % (i+1)]=hp.anafast(globals()['map13_region_%s' % (i+1)])
    #using the jackknife function to compute the bias of power spectrum above
    jack_knife=np.append(jack_knife,jackknife_stats(globals()['cl_13_region_%s' % (i+1)],test_statistic,0.95)[1])
print(jack_knife)
